In [1]:
import requests
import re
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
# Coloca tus credenciales aquí
CLIENT_ID = "9af49838e1b74e169724be1ab629e364"
CLIENT_SECRET = "33372bda1f89408abd8a1c9cfcc7804f"

# Autenticación
auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)


search(q, limit=10, offset=0, type='track', market=None)

    searches for an item

    Parameters:

            q - the search query (see how to write a query in the

                official documentation https://developer.spotify.com/documentation/web-api/reference/search/) # noqa

            limit - the number of items to return (min = 1, default = 10, max = 50). The limit is applied

                within each type, not on the total response.

            offset - the index of the first item to return

            type - the types of items to return. One or more of ‘artist’, ‘album’,

                ‘track’, ‘playlist’, ‘show’, and ‘episode’. If multiple types are desired, pass in a comma separated string; e.g., ‘track,album,episode’.

            market - An ISO 3166-1 alpha-2 country code or the string

                from_token.



ESTE NO HE SIDO CAPAZ DE ELABORARLO MÁS ALLÁ.

In [3]:
def buscar_año (q = '2022', limit = '50', offset = '1', type = 'track', market = 'ES'):
    
    result = sp.search (q =q, limit = limit, offset=offset, type = type,market = market)
    
    print (result)
    return result

CON ESTE HE CONSEGUIDO DATA FRAMES INTERESANTES. FALTARÍA FILTRAR POR FECHAS:

In [4]:
def rastreo_global(genero, limit=50, market='ES', max_results=200):
    """
    Realiza una búsqueda global de canciones de un género en Spotify.
    
    :param genero: Género musical a buscar (por defecto 'pop').
    :param limit: Número de canciones por solicitud (máx. 50).
    :param market: Mercado (por defecto 'ES').
    :param max_results: Número total de resultados deseados.
    :return: Lista de canciones con información relevante.
    """
    
    datos = []
    offset = 0  # Para la paginación

    while len(datos) < max_results:
        # Realizar la búsqueda en Spotify
        busqueda = sp.search(q=genero, limit=limit, offset=offset, type='track', market=market)
        
        # Obtener los resultados
        items = busqueda['tracks']['items']
        
        # Si no hay más resultados, salir del bucle
        if not items:
            break

        for item in items:
            track_info = {
                'track_name': item['name'],
                'artist_name': item['artists'][0]['name'],  # Primer artista
                'release_date': item['album']['release_date'],
                'id_track': item['id'],
            }
            datos.append(track_info)

            # Detenernos si alcanzamos el máximo de resultados
            if len(datos) >= max_results:
                break

        # Aumentamos el offset para la siguiente paginación
        offset += limit
    
    return datos

In [5]:
rastreo_global ('reaggeton')

[{'track_name': 'Reggaetón Lento (Bailemos)',
  'artist_name': 'CNCO',
  'release_date': '2016-08-26',
  'id_track': '3AEZUABDXNtecAOSC1qTfo'},
 {'track_name': 'La Novia Es Chiquita - Reaggeton Mix Extended Version',
  'artist_name': 'Melody',
  'release_date': '2004',
  'id_track': '3WJtmsnxsKYsLvWEw2NS4V'},
 {'track_name': 'FARDOS',
  'artist_name': 'JC Reyes',
  'release_date': '2024-02-16',
  'id_track': '33VIvWmmCUgix6Htm1WdOJ'},
 {'track_name': 'Reggaetonera',
  'artist_name': 'Anuel AA',
  'release_date': '2020-05-29',
  'id_track': '6yVkakCN460KOVol9ZazjH'},
 {'track_name': 'Sólo para ti (feat. Iván y Ab) - Includes Reaggeton',
  'artist_name': 'Face 2 Face',
  'release_date': '2006-05-22',
  'id_track': '0YmTO2PZHfKB8Uj2YCv2DX'},
 {'track_name': 'BADGYAL',
  'artist_name': 'SAIKO',
  'release_date': '2024-04-26',
  'id_track': '3ng8tfwvzR4BBwa9yaMms6'},
 {'track_name': 'Reggaeton',
  'artist_name': 'Beny Jr',
  'release_date': '2025-01-30',
  'id_track': '6wqreX8dJEvH5UadXoYx6

In [6]:
reggeaton = rastreo_global ('reggaeton')
df_reggeaton = pd.DataFrame (reggeaton)
df_reggeaton

,track_name,artist_name,release_date,id_track
0,Reggaetón Lento (Bailemos),CNCO,2016-08-26,3AEZUABDXNtecAOSC1qTfo
1,Reggaeton,Beny Jr,2025-01-30,6wqreX8dJEvH5UadXoYx6y
2,FARDOS,JC Reyes,2024-02-16,33VIvWmmCUgix6Htm1WdOJ
3,Reggaetonera,Anuel AA,2020-05-29,6yVkakCN460KOVol9ZazjH
4,UWAIE,Kapo,2024-08-15,5QjmUqgpPQgXgg4606DqZF
...,...,...,...,...
195,Reggaeton Beat 2023,David y Golia Beats,2023-01-26,0Ao5F195tNatheZzeNGlrT
196,58,JHAYCO,2024-09-05,0KnYZ9Y8HLEzIRaFz9UQ57
197,Reggaeton Lento,Jose Asunción,2017-07-14,1PKMLCOUZSNvQ79DGFSmaU
198,ROSA LÍO,JC Reyes,2023-05-28,3lPrBYTWQWSYj2P054WYHw


In [19]:
listado_final_reggeateon =[]

for f in reggeaton:
    
    if f ['release_date'][:4]== '2022':
        
        print (f)
        listado_final_reggeateon. append (f)
        
        print (listado_final_reggeateon)

{'track_name': 'Reggaetón Lento (Bailemos)', 'artist_name': 'CNCO', 'release_date': '2022-08-01', 'id_track': '5VlQvKIorEWqUedD3rl4yQ'}
[{'track_name': 'Reggaetón Lento (Bailemos)', 'artist_name': 'CNCO', 'release_date': '2022-08-01', 'id_track': '5VlQvKIorEWqUedD3rl4yQ'}]
{'track_name': 'Reggaetón', 'artist_name': 'Standly', 'release_date': '2022-11-11', 'id_track': '0MEUoxYlDHOSnNvtPL5MKN'}
[{'track_name': 'Reggaetón Lento (Bailemos)', 'artist_name': 'CNCO', 'release_date': '2022-08-01', 'id_track': '5VlQvKIorEWqUedD3rl4yQ'}, {'track_name': 'Reggaetón', 'artist_name': 'Standly', 'release_date': '2022-11-11', 'id_track': '0MEUoxYlDHOSnNvtPL5MKN'}]
{'track_name': 'Mujeriego', 'artist_name': 'Ryan Castro', 'release_date': '2022-08-05', 'id_track': '4jiYaQVHzMfKhEeLypWpNO'}
[{'track_name': 'Reggaetón Lento (Bailemos)', 'artist_name': 'CNCO', 'release_date': '2022-08-01', 'id_track': '5VlQvKIorEWqUedD3rl4yQ'}, {'track_name': 'Reggaetón', 'artist_name': 'Standly', 'release_date': '2022-11-

In [20]:
type(listado_final_reggeateon)

list

In [21]:
df_listado_final_reggaeton = pd. DataFrame (listado_final_reggeateon)
df_listado_final_reggaeton

,track_name,artist_name,release_date,id_track
0,Reggaetón Lento (Bailemos),CNCO,2022-08-01,5VlQvKIorEWqUedD3rl4yQ
1,Reggaetón,Standly,2022-11-11,0MEUoxYlDHOSnNvtPL5MKN
2,Mujeriego,Ryan Castro,2022-08-05,4jiYaQVHzMfKhEeLypWpNO
3,Reggaetón,Dímelo Flow,2022-06-09,3mGDp8S3l1X5dymWzaCYfv
4,Reggaeton,Kaliza,2022-10-21,1RdK0uDzrMt3oXVHpTahAD
5,Reggaeton,Kaliza,2022-06-15,00nSBP5WwbgX2geqdn3bpt
6,Se Enamoró de un Reggaetonero,Alejo,2022-06-22,0dlEA0Yhxsd09QWncDvaXF
7,Reggaeton,Lapiz Conciente,2022-09-07,12V4x9jLu1WdwRhfbu1yvm
8,Bailemos Reggaeton,Standly,2022-03-17,0YRgRpM2bdmQEkiI8bWDBA
9,IBIZA,JC Reyes,2022-06-01,1Z1zLwWRLVxFcpM8NypchZ


In [7]:
rastreo_global ('pop')

[{'track_name': 'Popular',
  'artist_name': 'Ariana Grande',
  'release_date': '2024-11-22',
  'id_track': '3i8fIJNV0yuC4kXT3SkVBZ'},
 {'track_name': 'Popular (with Playboi Carti & Madonna) - From The Idol Vol. 1 (Music from the HBO Original Series)',
  'artist_name': 'The Weeknd',
  'release_date': '2023-06-02',
  'id_track': '6WzRpISELf3YglGAh7TXcG'},
 {'track_name': 'Dior',
  'artist_name': 'Pop Smoke',
  'release_date': '2019-07-26',
  'id_track': '79s5XnCN4TJKTVMSmOx8Ep'},
 {'track_name': 'Pop',
  'artist_name': 'La Oreja de Van Gogh',
  'release_date': '2000-09-07',
  'id_track': '4FXSEdmbbeGmPIQltH27C8'},
 {'track_name': 'Popurrí de Rumbas',
  'artist_name': 'Sal Gorda',
  'release_date': '2014-02-19',
  'id_track': '0DBb3FibybuYub7OzLR69i'},
 {'track_name': 'Poporopa',
  'artist_name': 'Morad',
  'release_date': '2023-05-12',
  'id_track': '4Fv2ZAQ0U1lsEGjhghcdb0'},
 {'track_name': 'Invincible',
  'artist_name': 'Pop Smoke',
  'release_date': '2020-02-07',
  'id_track': '792Hwh

In [8]:
pop = rastreo_global ('pop')
df_pop = pd.DataFrame (pop)
df_pop

,track_name,artist_name,release_date,id_track
0,Popular,Ariana Grande,2024-11-22,3i8fIJNV0yuC4kXT3SkVBZ
1,Popular (with Playboi Carti & Madonna) - From ...,The Weeknd,2023-06-02,6WzRpISELf3YglGAh7TXcG
2,Dior,Pop Smoke,2019-07-26,79s5XnCN4TJKTVMSmOx8Ep
3,Pop,La Oreja de Van Gogh,2000-09-07,4FXSEdmbbeGmPIQltH27C8
4,Popurrí de Rumbas,Sal Gorda,2014-02-19,0DBb3FibybuYub7OzLR69i
...,...,...,...,...
195,Cups (Pitch Perfect’s “When I’m Gone”) - Pop V...,Anna Kendrick,2013-01-01,2sCl0FFOzZSmYZs90zOpd1
196,Popurrí (Mi Negra / La Rosita / Le Tocaron Gua...,Tordillo Negro,2018-07-12,2g4haGjXF5Z6JxJYE1IFs6
197,Azul,J Balvin,2023-06-14,04ao9P1v2BACYCaCGU8XMK
198,Censor (feat. Popcaan & IRAH),Chase & Status,2022-06-10,18EcR5oRBNKqKqX9qxpcKC


In [22]:
listado_final_pop =[]

for f in pop:
    
    if f ['release_date'][:4]== '2022':
        
        print (f)
        listado_final_pop. append (f)
        
        print (listado_final_pop)

{'track_name': 'POP!', 'artist_name': 'NAYEON', 'release_date': '2022-06-24', 'id_track': '3lOMJTQTd6J34faYwASc33'}
[{'track_name': 'POP!', 'artist_name': 'NAYEON', 'release_date': '2022-06-24', 'id_track': '3lOMJTQTd6J34faYwASc33'}]
{'track_name': 'Pop Muzik', 'artist_name': 'M', 'release_date': '2022-06-03', 'id_track': '6z3HAUZpAyJ0ctsbAwAiw3'}
[{'track_name': 'POP!', 'artist_name': 'NAYEON', 'release_date': '2022-06-24', 'id_track': '3lOMJTQTd6J34faYwASc33'}, {'track_name': 'Pop Muzik', 'artist_name': 'M', 'release_date': '2022-06-03', 'id_track': '6z3HAUZpAyJ0ctsbAwAiw3'}]
{'track_name': 'Pop Star', 'artist_name': 'Coco & Clair Clair', 'release_date': '2022-11-04', 'id_track': '0widrZ6KVNuIPhbM1rWPDR'}
[{'track_name': 'POP!', 'artist_name': 'NAYEON', 'release_date': '2022-06-24', 'id_track': '3lOMJTQTd6J34faYwASc33'}, {'track_name': 'Pop Muzik', 'artist_name': 'M', 'release_date': '2022-06-03', 'id_track': '6z3HAUZpAyJ0ctsbAwAiw3'}, {'track_name': 'Pop Star', 'artist_name': 'Coco 

In [23]:
df_listado_final_pop = pd. DataFrame (listado_final_pop)
df_listado_final_pop

,track_name,artist_name,release_date,id_track
0,POP!,NAYEON,2022-06-24,3lOMJTQTd6J34faYwASc33
1,Pop Muzik,M,2022-06-03,6z3HAUZpAyJ0ctsbAwAiw3
2,Pop Star,Coco & Clair Clair,2022-11-04,0widrZ6KVNuIPhbM1rWPDR
3,Popeye - Remix Version,Sersy 23,2022-12-22,5lg1j7ONNLILWCXIvYpT5H
4,Poppin,Yeat,2022-02-18,3wn8nOygkHLUQ9dlXM1rKW
5,NIGHT DANCER,imase,2022-11-09,3gmn76fCtpDazGMnN0QMHX
6,Temptation (feat. Poppy Baskcomb),Jess Bays,2022-02-18,3fvsxmytTns1ApIWBqfANZ
7,Mira Que Linda,poperulin,2022-12-26,0OGANwLNf20iwK38AbIg4D
8,Pop Out (feat. Nardo Wick),Lil Baby,2022-10-14,0VOMBffX1llDu37yQUt3u3
9,Popcorn Salé,SANTA,2022-03-25,4MpqiN2moRQR2t0DO43MZH


In [9]:
rastreo_global('rap')

[{'track_name': 'RAPAPA',
  'artist_name': 'JC Reyes',
  'release_date': '2024-10-18',
  'id_track': '79j3Cl3AEAEhXK9cjjfd02'},
 {'track_name': 'Rapido (with Cris Mj)',
  'artist_name': 'Dei V',
  'release_date': '2024-06-14',
  'id_track': '1emJPywtseFLmLCcPY92dW'},
 {'track_name': 'Rap God',
  'artist_name': 'Eminem',
  'release_date': '2013-11-05',
  'id_track': '6or1bKJiZ06IlK0vFvY75k'},
 {'track_name': 'RÁPIDO',
  'artist_name': 'Mora',
  'release_date': '2023-04-27',
  'id_track': '6peVAUzSxDlTigQw5T99O5'},
 {'track_name': '#RapSinCorte L',
  'artist_name': 'Foyone',
  'release_date': '2021-11-12',
  'id_track': '1POeUrfkWVRPjaUbf5fgca'},
 {'track_name': 'Rapeadita LOVE',
  'artist_name': 'Rels B',
  'release_date': '2024-04-26',
  'id_track': '1mqixStNxcORNYvXSw2uXp'},
 {'track_name': 'Rapido',
  'artist_name': 'Amenazzy',
  'release_date': '2021-06-18',
  'id_track': '4OMeEpkakI4Vb77gulgTDw'},
 {'track_name': 'The Rapture Pt.III',
  'artist_name': '&ME',
  'release_date': '2023

In [10]:
rap = rastreo_global ('rap')
df_rap = pd.DataFrame (rap)
df_rap

,track_name,artist_name,release_date,id_track
0,RAPAPA,JC Reyes,2024-10-18,79j3Cl3AEAEhXK9cjjfd02
1,Rapido (with Cris Mj),Dei V,2024-06-14,1emJPywtseFLmLCcPY92dW
2,Rap God,Eminem,2013-11-05,6or1bKJiZ06IlK0vFvY75k
3,RÁPIDO,Mora,2023-04-27,6peVAUzSxDlTigQw5T99O5
4,#RapSinCorte L,Foyone,2021-11-12,1POeUrfkWVRPjaUbf5fgca
...,...,...,...,...
195,"Nocturnes, Op. 32: II. Lento (Arr. for Cello &...",Frédéric Chopin,2025-02-20,5lwVj1rtkirHGMeGAk41KO
196,Someday My Prince Will Come - Cello-Piano Version,Frank Churchill,2025-02-14,350FDBQ3siAypjz2stp69j
197,"Alcina, HWV 34: Verdi prati (Arr. for Cello & ...",George Frideric Handel,2025-02-16,3IJnMoWqql2QXILGCVpYvD
198,"Joy (After Bach's Jesu, Joy of Man's Desiring,...",Ludwig van Beethoven,2025-02-20,6YDS9E5NDHQt5LTqYWVkG2


In [24]:
listado_final_rap =[]

for f in rap:
    
    if f ['release_date'][:4]== '2022':
        
        print (f)
        listado_final_rap. append (f)
        
        print (listado_final_rap)

{'track_name': 'RAPTURE', 'artist_name': 'INTERWORLD', 'release_date': '2022-07-01', 'id_track': '3thB8qaxyKtLJ7ao1NpkRB'}
[{'track_name': 'RAPTURE', 'artist_name': 'INTERWORLD', 'release_date': '2022-07-01', 'id_track': '3thB8qaxyKtLJ7ao1NpkRB'}]
{'track_name': 'rápido lento', 'artist_name': 'Emilia', 'release_date': '2022-05-31', 'id_track': '3Tx1gnGvzBYRtEnr3o705w'}
[{'track_name': 'RAPTURE', 'artist_name': 'INTERWORLD', 'release_date': '2022-07-01', 'id_track': '3thB8qaxyKtLJ7ao1NpkRB'}, {'track_name': 'rápido lento', 'artist_name': 'Emilia', 'release_date': '2022-05-31', 'id_track': '3Tx1gnGvzBYRtEnr3o705w'}]
{'track_name': 'Te Pone Bien - Con Kase.O', 'artist_name': 'Sho-Hai', 'release_date': '2022-07-22', 'id_track': '2PeW2EAbZLL8kt7vDsSL9V'}
[{'track_name': 'RAPTURE', 'artist_name': 'INTERWORLD', 'release_date': '2022-07-01', 'id_track': '3thB8qaxyKtLJ7ao1NpkRB'}, {'track_name': 'rápido lento', 'artist_name': 'Emilia', 'release_date': '2022-05-31', 'id_track': '3Tx1gnGvzBYRtEnr

In [25]:
df_listado_final_rap = pd.DataFrame (listado_final_rap)
df_listado_final_rap

,track_name,artist_name,release_date,id_track
0,RAPTURE,INTERWORLD,2022-07-01,3thB8qaxyKtLJ7ao1NpkRB
1,rápido lento,Emilia,2022-05-31,3Tx1gnGvzBYRtEnr3o705w
2,Te Pone Bien - Con Kase.O,Sho-Hai,2022-07-22,2PeW2EAbZLL8kt7vDsSL9V
3,Wild Flower (with youjeen),RM,2022-12-02,1AZjCZSGEjlxUufFQwSszF
4,Wrapped Around Your Finger,Post Malone,2022-06-03,6Kfoo60npYPdvNxMPMiDpX
5,RAPPEL,Gazo,2022-07-01,48BdA5qBxXVoe59xvu0bwc
6,Rap,JALEO,2022-05-17,3ZxuXpr8rCiyMV84eDJ6HC
7,RAP,PAR,2022-09-19,59OrzxRvbgKFwozKkzmVES
8,Rap,Aschkobar,2022-08-31,5LlJ6YsaxKJWS3Lpa3vf5A
9,RAP,Jura,2022-06-13,6ZwvUhm7D4Nku2PjyCSSAx


In [11]:
rastreo_global ('indie')

[{'track_name': 'Indietro',
  'artist_name': 'Tiziano Ferro',
  'release_date': '2015-11-20',
  'id_track': '6wqmc9hgPiUxAi9qk75nzm'},
 {'track_name': 'Indie - Original Mix',
  'artist_name': 'Korpus',
  'release_date': '2014-09-12',
  'id_track': '2CpBD0NWTQezAW8nWupSIz'},
 {'track_name': 'Indie esperanzador',
  'artist_name': 'Jimena Amarillo',
  'release_date': '2022-07-28',
  'id_track': '0qYoAYpwQr3g09RfzlqSEG'},
 {'track_name': 'Indie',
  'artist_name': 'Retro 1312',
  'release_date': '2023-07-21',
  'id_track': '0NI5r3Kc6WZL8XHz0DaAip'},
 {'track_name': 'Indie',
  'artist_name': 'Ost & Kjex',
  'release_date': '2023-04-14',
  'id_track': '7rSuqMaEloVncVc8RDdoDX'},
 {'track_name': 'Indie Rokkers',
  'artist_name': 'MGMT',
  'release_date': '2005-08-30',
  'id_track': '0gCNIZWff5ymNGYMbwImMz'},
 {'track_name': 'Indie',
  'artist_name': 'D 4 Ma',
  'release_date': '2003-09-09',
  'id_track': '1VWB9XU2ggrjMlspo0xVEP'},
 {'track_name': 'Indie Pop Type Beat',
  'artist_name': 'Billy M

In [12]:
indie = rastreo_global ('indie')
df_indie = pd. DataFrame (indie)
df_indie

,track_name,artist_name,release_date,id_track
0,Indietro,Tiziano Ferro,2015-11-20,6wqmc9hgPiUxAi9qk75nzm
1,Indie - Original Mix,Korpus,2014-09-12,2CpBD0NWTQezAW8nWupSIz
2,Indie esperanzador,Jimena Amarillo,2022-07-28,0qYoAYpwQr3g09RfzlqSEG
3,Indie,Retro 1312,2023-07-21,0NI5r3Kc6WZL8XHz0DaAip
4,Indie,Ost & Kjex,2023-04-14,7rSuqMaEloVncVc8RDdoDX
...,...,...,...,...
195,Take Me There,RXPHY,2024-07-01,4lbjXsKqDHGZmz9R5dmOje
196,White Mice,Mo-Dettes,2011-06-13,6QHZjDhY00tTwJWOna8Vyb
197,Indie Boy,Los Colorados,2012,4UQlSooMVKxYsmsKrAWGpn
198,Indietro,DJ Shocca,2013-12-02,3YymRpgQnFRMROu6OXjWbI


In [26]:
listado_final_indie =[]

for f in indie:
    
    if f ['release_date'][:4]== '2022':
        
        print (f)
        listado_final_indie. append (f)
        
        print (listado_final_indie)

{'track_name': 'Indie esperanzador', 'artist_name': 'Jimena Amarillo', 'release_date': '2022-07-28', 'id_track': '0qYoAYpwQr3g09RfzlqSEG'}
[{'track_name': 'Indie esperanzador', 'artist_name': 'Jimena Amarillo', 'release_date': '2022-07-28', 'id_track': '0qYoAYpwQr3g09RfzlqSEG'}]
{'track_name': "L'été indien", 'artist_name': 'Joe Dassin', 'release_date': '2022-11-04', 'id_track': '1Ggj10R2dFrBj9LjQscvDz'}
[{'track_name': 'Indie esperanzador', 'artist_name': 'Jimena Amarillo', 'release_date': '2022-07-28', 'id_track': '0qYoAYpwQr3g09RfzlqSEG'}, {'track_name': "L'été indien", 'artist_name': 'Joe Dassin', 'release_date': '2022-11-04', 'id_track': '1Ggj10R2dFrBj9LjQscvDz'}]
{'track_name': 'Indie Folk Acoustic', 'artist_name': 'Art Music', 'release_date': '2022-06-24', 'id_track': '5OgoALxAsdifEKhVeGt3yu'}
[{'track_name': 'Indie esperanzador', 'artist_name': 'Jimena Amarillo', 'release_date': '2022-07-28', 'id_track': '0qYoAYpwQr3g09RfzlqSEG'}, {'track_name': "L'été indien", 'artist_name': '

In [27]:
df_listado_final_indie = pd.DataFrame (listado_final_indie)
df_listado_final_indie

,track_name,artist_name,release_date,id_track
0,Indie esperanzador,Jimena Amarillo,2022-07-28,0qYoAYpwQr3g09RfzlqSEG
1,L'été indien,Joe Dassin,2022-11-04,1Ggj10R2dFrBj9LjQscvDz
2,Indie Folk Acoustic,Art Music,2022-06-24,5OgoALxAsdifEKhVeGt3yu
3,West Indies,Original Koffee,2022-03-25,2OFvClbKLxtHeVMb3O1pOX
4,Indie,Amaury Atrack,2022-09-09,6QNrqwHoaMPS4gcvfe7NBR
5,Indiesong,CFCF,2022-04-13,1LQisz6YTPRLDnRdW8mjen
6,Indie,Timewise,2022-01-28,0WGLoQgcp4VtyHDtJ6t5xK
7,indie,Jasper,2022-01-14,1Dp32GDsJzdrKTSqj6dsbr
8,Send Me Love,Beatkid,2022-07-26,5FRJxBNDa7obUqNSd97FKn
9,Indie Alt Boys,Ghostryder,2022-05-06,7JlVmQxB13lLM2mqfDa5fu
